In [ ]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt
from task1_data_preprocessing import preprocess_data
%matplotlib inline

data = preprocess_data()
prices = data["pivot_close"][['TSLA','BND','SPY']].sort_index()
daily_returns = prices.pct_change().dropna()


In [ ]:
ANNUAL_DAYS = 252
hist_mean_daily = daily_returns.mean()
hist_annual = hist_mean_daily * ANNUAL_DAYS

# load forecast CSV (choose LSTM or ARIMA)
df_forecast = pd.read_csv("task3_outputs/forecast_arima_6m.csv", parse_dates=['date'], index_col='date')
last_price = prices['TSLA'].iloc[-1]
p_h = df_forecast['arima_forecast'].iloc[-1]
horizon = len(df_forecast)
tsla_exp = (p_h/last_price -1 + 1)**(ANNUAL_DAYS/horizon) - 1  # simplified; use function in script
print("TSLA derived expected annual return:", tsla_exp)

mu = pd.Series({"TSLA": tsla_exp, "BND":hist_annual['BND'], "SPY":hist_annual['SPY']})
S = daily_returns[['TSLA','BND','SPY']].cov() * ANNUAL_DAYS
print(mu); print(S)


In [ ]:
from pypfopt import EfficientFrontier
ef = EfficientFrontier(mu, S)
w_sh = ef.max_sharpe(risk_free_rate=0.03)
w_sh = ef.clean_weights()
perf_sh = ef.portfolio_performance(risk_free_rate=0.03, verbose=True)

ef2 = EfficientFrontier(mu, S)
w_mv = ef2.min_volatility()
w_mv = ef2.clean_weights()
perf_mv = ef2.portfolio_performance(risk_free_rate=0.03, verbose=True)


In [ ]:
# Cell 4 — Plot Efficient Frontier, mark Max Sharpe & Min Vol portfolios
import numpy as np
import matplotlib.pyplot as plt
from pypfopt import EfficientFrontier
%matplotlib inline

# Risk-free rate (adjust as needed)
RISK_FREE_RATE = 0.03

# 1) Compute Max Sharpe portfolio
ef_sharpe = EfficientFrontier(mu, S)
w_sharpe = ef_sharpe.max_sharpe(risk_free_rate=RISK_FREE_RATE)
w_sharpe = ef_sharpe.clean_weights()
ret_sh, vol_sh, sr_sh = ef_sharpe.portfolio_performance(risk_free_rate=RISK_FREE_RATE)

# 2) Compute Min Volatility portfolio
ef_min = EfficientFrontier(mu, S)
w_min = ef_min.min_volatility()
w_min = ef_min.clean_weights()
ret_min, vol_min, sr_min = ef_min.portfolio_performance(risk_free_rate=RISK_FREE_RATE)

# 3) Build efficient frontier points (sampled)
target_returns = np.linspace(mu.min(), mu.max(), 50)
frontier_rets = []
frontier_vols = []

for target in target_returns:
    try:
        ef_tmp = EfficientFrontier(mu, S)
        ef_tmp.efficient_return(target_return=target)
        ret, vol, _ = ef_tmp.portfolio_performance(risk_free_rate=RISK_FREE_RATE)
        frontier_rets.append(ret)
        frontier_vols.append(vol)
    except Exception:
        # some target returns may be infeasible; skip them
        continue

# 4) Plot
plt.figure(figsize=(10, 6))
plt.plot(frontier_vols, frontier_rets, 'b--', lw=1.5, label='Efficient Frontier')

# Plot asset points
for asset in mu.index:
    asset_vol = np.sqrt(S.loc[asset, asset])
    asset_ret = mu.loc[asset]
    plt.scatter(asset_vol, asset_ret, s=90, label=f"{asset} (asset)")

# Mark Max Sharpe and Min Vol points
plt.scatter(vol_sh, ret_sh, c='green', marker='*', s=220, label='Max Sharpe')
plt.scatter(vol_min, ret_min, c='red', marker='*', s=220, label='Min Volatility')

plt.xlabel('Annualized Volatility')
plt.ylabel('Annualized Return')
plt.title('Efficient Frontier — TSLA (forecast) + BND & SPY (historical)')
plt.legend(loc='best', fontsize='small')
plt.grid(alpha=0.3)
plt.tight_layout()

# Optional: save the figure
plt.savefig("task4_outputs/efficient_frontier_plot.png", dpi=300)
plt.show()

# 5) Print numeric summaries (nice formatting)
def fmt_weights(w):
    return {k: round(v*100, 2) for k, v in w.items()}

print("\nMax Sharpe Portfolio")
print("--------------------")
print("Weights (%):", fmt_weights(w_sharpe))
print(f"Expected annual return: {ret_sh:.4f}")
print(f"Annual volatility: {vol_sh:.4f}")
print(f"Sharpe Ratio: {sr_sh:.4f}")

print("\nMin Volatility Portfolio")
print("------------------------")
print("Weights (%):", fmt_weights(w_min))
print(f"Expected annual return: {ret_min:.4f}")
print(f"Annual volatility: {vol_min:.4f}")
print(f"Sharpe Ratio: {sr_min:.4f}")
